In [1]:
# importing all necessary libraries
import numpy as np
import pandas as pd
import random
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files
path = "/content/drive/MyDrive/Monarch/RL/Tic Tac Toe/transition_probability_list.csv"
pmt = pd.read_csv(path)
#Create transition probability dictionary from imported csv file
probability_transition_dict = {}
for i in range(len(pmt)):
  first_state = pmt.iloc[i, 1][2:11]
  second_state = pmt.iloc[i, 1][15:24]

  probability_transition_dict[first_state, second_state] = pmt.iloc[i, 2]

Mounted at /content/drive


In [7]:
# Creates an empty board
def create_board():
    return(np.array([[0, 0, 0],
                     [0, 0, 0],
                     [0, 0, 0]]))

# Check for empty places on board
def possibilities(board):
    l = []

    for i in range(len(board)):
        for j in range(len(board)):

            if board[i][j] == 0:
                l.append((i, j))
    return(l)

# Select a random place for the player
def random_place(board, player):
    selection = possibilities(board)
    current_loc = random.choice(selection)
    board[current_loc] = player



    return(board)

#Select the best move using the probability transition matrix after training
def best_move(board, player):

  selections = possibilities(board)
  optimal_move_score = 0
  optimal_move = selections[0]

  for selection in selections:
    new_board = board.copy()
    new_board[selection] = player

    board_string = numpy_array_to_string(board)
    new_board_string = numpy_array_to_string(new_board)
    if (board_string, new_board_string) not in probability_transition_dict:
      #print("unencountered state")
      probability_transition_dict[(board_string, new_board_string)] = 0

    current_move_score = probability_transition_dict[board_string, new_board_string]
    #print(new_board)
    #print(current_move_score)
    #print(current_move_score)
    if current_move_score > optimal_move_score:
      optimal_move_score = current_move_score
      optimal_move = selection

  board[optimal_move] = player
  return board


# Checks whether the player has three of their marks in a horizontal row
def row_win(board, player):
    for x in range(len(board)):
        win = True

        for y in range(len(board)):
            if board[x, y] != player:
                win = False
                continue

        if win == True:
            return(win)
    return(win)

# Checks whether the player has three of their marks in a vertical row
def col_win(board, player):
    for x in range(len(board)):
        win = True

        for y in range(len(board)):
            if board[y][x] != player:
                win = False
                continue

        if win == True:
            return(win)
    return(win)

# Checks whether the player has three of their marks in a diagonal row
def diag_win(board, player):
    win = True
    y = 0
    for x in range(len(board)):
        if board[x, x] != player:
            win = False
    if win:
        return win
    win = True
    if win:
        for x in range(len(board)):
            y = len(board) - 1 - x
            if board[x, y] != player:
                win = False
    return win

# Evaluates whether there is a winner or a tie
def evaluate(board, state_list):
    winner = 0

    for player in [1, 2]:
        if (row_win(board, player) or
                col_win(board, player) or
                diag_win(board, player)):
            winner = player
            if len(state_list) > 1:
              update_probability_transition_dictionary(state_list, winner)
    if np.all(board != 0) and winner == 0:
        winner = -1
    return winner

#takes a 2-d numpy array and create a string representation
def numpy_array_to_string(array):
  string_array = ""
  for row in array:
    for element in row:
      string_array += str(element)
  return string_array

#This function is for training (creating a new transition dictionary)
#Change the training parameter to True
def update_probability_transition_dictionary(state_list, winner, training = False):
    #In case of a tie, do not update anything (might change this later)
    if winner == -1:
      return
    print(state_list)

    if training:
      if winner == 1:
        i = 0
        while i <= len(state_list)-1:
          x = state_list[i]
          y = state_list[i+1]
          if (x, y) not in probability_transition_dict:
            probability_transition_dict[(x, y)] = 1
          else:
            probability_transition_dict[(x, y)] += 1
          i += 2
      else:
        i = 1
        while i <= len(state_list)-1:
          x = state_list[i]
          y = state_list[i+1]
          if (x, y) not in probability_transition_dict:
            probability_transition_dict[(x, y)] = 1
          else:
            probability_transition_dict[(x, y)] += 1
          i += 2

# Main function to play the game
def play_game():

    board, winner, counter = create_board(), 0, 1
    print(board)
    state_list = []
    state_list.append(numpy_array_to_string(board))
    # player_number_decision = random.randint(0, 1)
    # if player_number_decision == 0:
    #    player1 = 1
    #    player2 = 2
    # else:
    #    player1 = 2
    #    player2 = 1
    #start = time.time()
    player1 = 1
    player2 = 2
    while winner == 0:
        for player in [player1, player2]:
            board = random_place(board, player)
            state_list.append(numpy_array_to_string(board))
            print("Board after " + str(counter) + " move")
            print(board)
            #sleep(2)
            counter += 1
            if counter > 4:
                winner = evaluate(board, state_list)
            if winner != 0:
                break
    #end = time.time()
    #print("Game over. It took "+ str(end-start) + " seconds")
    return(winner)

# Main function for bot to play with human
def play_game_w_human():
    player_number = int(input("Welcome human to tic tac toe: enter player number (1 or 2) "))

    board, winner, counter = create_board(), 0, 1
    print(board)
    state_list = []
    state_list.append(numpy_array_to_string(board))


    human = 1
    computer = 2
    player_list = [human, computer]
    if player_number == 2:
        human = 2
        computer = 1
        player_list = [computer, human]

    #start = time.time()
    while winner == 0:
        for player in player_list:
          if player == human:
            player_move = input("Enter your next move: ")
            player_move = player_move.split()
            player_move = (int(player_move[0]), int(player_move[1]))
            board[player_move] = player
          else:
            print("best_move_cpu")
            board = best_move(board, player)
            print("Board after " + str(counter) + " move")
            print(board)
          counter += 1
          # if counter > 4:
          winner = evaluate(board, state_list)
          if winner != 0:
            break
    #end = time.time()
    if winner == 1 or winner == 2:
      print("Game over. " + str(winner) + " is the winner")
    else:
      print("The game is a tie!")

    return



In [ ]:
# Trial Driver Code
# Done 2000000 trials
# Trials took 263 minutes
# 1746321 wins out of 2000000 trials (player 1 or player 2 won)

start = time.time()
n = 2000000
counter = 0
for i in range(n):
  winner = play_game()
  if winner != -1:
    counter += 1
  print(i)
end = time.time()
print("Trials took " + str(end-start) + " seconds")
print(str(counter) + " wins out of " + str(n) + " trials")



In [12]:
#Sanity Checks/Debugging

#Empty board
blank_board = numpy_array_to_string(np.zeros((3, 3), dtype=int))

#first move should always go in the middle
middle_mid = numpy_array_to_string(np.array([[0, 0, 0], [0, 1, 0], [0, 0, 0]]))

#first move to diagonal probabilities should all be within a delta
top_left = numpy_array_to_string(np.array([[1, 0, 0], [0, 0, 0], [0, 0, 0]]))
top_right = numpy_array_to_string(np.array([[0, 0, 1], [0, 0, 0], [0, 0, 0]]))
bottom_left = numpy_array_to_string(np.array([[0, 0, 0], [0, 0, 0], [1, 0, 0]]))
bottom_right = numpy_array_to_string(np.array([[0, 0, 0], [0, 0, 0], [0, 0, 1]]))

#first mode to middle edge probabilities should all be within a delta
top_mid = numpy_array_to_string(np.array([[0, 1, 0], [0, 0, 0], [0, 0, 0]]))
left_mid = numpy_array_to_string(np.array([[0, 0, 0], [1, 0, 0], [0, 0, 0]]))
right_mid = numpy_array_to_string(np.array([[0, 0, 0], [0, 0, 1], [0, 0, 0]]))
bottom_mid = numpy_array_to_string(np.array([[0, 0, 0], [0, 0, 0], [0, 1, 0]]))


In [15]:
print("middle 1 score: " + str(probability_transition_dict[(blank_board, middle_mid)]))
print("")

print("diagonal move scores")
print("top left: " + str(probability_transition_dict[(blank_board, top_left)]))
print("top right: " + str(probability_transition_dict[(blank_board, top_right)]))
print("bottom left: " + str(probability_transition_dict[(blank_board, bottom_left)]))
print("bottom right: " + str(probability_transition_dict[(blank_board, bottom_right)]))
print("")

print("Middle edge move scores")
print("top mid: " + str(probability_transition_dict[(blank_board, top_mid)]))
print("left mid: " + str(probability_transition_dict[(blank_board, left_mid)]))
print("right mid: " + str(probability_transition_dict[(blank_board, right_mid)]))
print("bottom mid: " + str(probability_transition_dict[(blank_board, bottom_mid)]))

# print("Sample scores from a game played with a human")
# first_state = numpy_array_to_string(np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]]))
# second_state = numpy_array_to_string(np.array([[1, 0, 0], [0, 0, 0], [0, 0, 0]]))
# third_state = numpy_array_to_string(np.array([[1, 0, 0], [0, 2, 0], [0, 0, 0]]))
# fourth_state = numpy_array_to_string(np.array([[1, 0, 1], [0, 2, 0], [0, 0, 0]]))
# fifth_state = numpy_array_to_string(np.array([[1, 2, 1], [0, 2, 0], [0, 0, 0]]))
# sixth_state = numpy_array_to_string(np.array([[1, 2, 1], [1, 2, 0], [0, 0, 0]]))
# seventh_state = numpy_array_to_string(np.array([[1, 2, 1], [1, 2, 0], [0, 2, 0]]))


# print("first score: " + str(probability_transition_dict[(first_state, second_state)]))
# print("second score: " + str(probability_transition_dict[(second_state, third_state)]))
# print("third score: " + str(probability_transition_dict[(third_state, fourth_state)]))
# print("fourth score: " + str(probability_transition_dict[(fourth_state, fifth_state)]))
# print("fifth score: " + str(probability_transition_dict[(fifth_state, sixth_state)]))
# print("sixth score: " + str(probability_transition_dict[(sixth_state, seventh_state)]))



middle 1 score: 154840

diagonal move scores
top left: 134797
top right: 134989
bottom left: 134982
bottom right: 134489

Middle edge move scores
top mid: 119080
left mid: 118498
right mid: 119020
bottom mid: 118755


In [8]:
 #run with human input
play_game_w_human()



Welcome human to tic tac toe: enter player number (1 or 2) 2
[[0 0 0]
 [0 0 0]
 [0 0 0]]
best_move_cpu
Board after 1 move
[[0 0 0]
 [0 1 0]
 [0 0 0]]
Enter your next move: 0 0
best_move_cpu
Board after 3 move
[[2 0 0]
 [0 1 0]
 [1 0 0]]
Enter your next move: 0 2
best_move_cpu
Board after 5 move
[[2 1 2]
 [0 1 0]
 [1 0 0]]
Enter your next move: 2 1
best_move_cpu
Board after 7 move
[[2 1 2]
 [1 1 0]
 [1 2 0]]
Enter your next move: 1 2
best_move_cpu
Board after 9 move
[[2 1 2]
 [1 1 2]
 [1 2 1]]
The game is a tie!


In [18]:
#Download the dictionary so that there's no need to retrain
probability_list = []
for key in probability_transition_dict.keys():
  probability_list.append([key,probability_transition_dict[key]])

probability_list_official = pd.DataFrame(probability_list)
probability_list_official.to_csv('transition_probability_list.csv')
files.download('transition_probability_list.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>